### LLM_Chain

使用本地的开源模型，利用ollama来运行llama2

确保ollama服务处于运行状态

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

接下来我们就可以尝试使用llm调用llama2模型了。我们问它"LangSmith"是什么。
这是训练数据中没有出现过的东西，所有它并不能给出很好的回答

In [ ]:
llm.invoke("how can langsmith help with testing?")

我们也可以使用提示模板来指导模型作出响应。提示模版的作用是，将用户的原始输入转换为对于llm更好的输入。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

现在我们可以将这些组合成一个简单的LLM链

In [ ]:
chain = prompt | llm 

我们现在可以调用它并问同样的问题。虽然它仍然不知道答案，但能以一种更适合技术作家的语气来回应!

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

ChatModel的输出(也是这个链的输出)是一条消息。然而，使用字符串通常要方便得多。让我们添加一个简单的输出解析器，将聊天消息转换为字符串。

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

现在可以把这个添加到前面的链中:

In [ ]:
chain = prompt | llm | output_parser

现在可以调用它并问同样的问题。现在答案将是一个字符串(而不是ChatMessage)。·

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

### Retrieval_Chain

为了正确回答最初的问题(“langsmith如何帮助测试?”)，我们需要为LLM提供额外的知识。可以通过检索来实现。当有太多的数据需要直接传递给LLM时，检索非常有用。然后，可以使用检索器只获取最相关的部分并将它们传递给LLM。在这个过程中，我们将查找相关文档，然后将它们传递给提示词。检索器可以由任何东西支持，比如SQL表、互联网等。但在本例中，我们将填充一个向量存储并将其用作检索器。

首先，我们需要加载要建立索引的数据。为此，我们将使用WebBaseLoader。这需要安装BeautifulSoup

In [ ]:
pip install beautifulsoup4

之后，我们可以导入和使用WebBaseLoader。

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

接下来，我们需要将其索引到vectorstore中。这需要几个组件，即embedding模型和vectorstore。

对于嵌入模型，我们再次提供了通过API或运行本地模型进行访问的示例。

确保Ollama正在运行(与LLM的设置相同)。

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

现在，我们可以使用这个嵌入模型将文档存储到vectorstore中。为了简单起见，我们将使用一个简单的本地矢量库FAISS。


首先，我们需要安装所需的软件包

In [ ]:
pip install faiss-cpu

然后我们可以建立索引

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

现在我们已经在vectorstore中为这些数据建立了索引，我们将创建一个检索链。该链将接收传入的问题，查找相关文档，然后将这些文档与原始问题一起传递给LLM，并要求它回答原始问题。

首先，让我们建立一个链，它接受一个问题和检索到的文档，并生成一个答案。


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

如果需要，我们可以通过直接传递文档来运行这个

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

然而，我们希望文档首先来自我们刚刚设置的检索器。这样，我们可以使用检索器动态地选择最相关的文档，并为给定的问题传递这些文档。

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

现在我们可以调用这个链。这将返回一个字典，来自LLM的响应在答案键中

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

这个答案应该准确得多

### Conversation Retrieval Chain

The chain we've created so far can only answer single questions. One of the main types of LLM applications that people are building are chat bots. So how do we turn this chain into one that can answer follow up questions?

We can still use the create_retrieval_chain function, but we need to change two things:

1.The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.

2.The final LLM chain should likewise take the whole history into account

#### Updating Retrieval

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (input) and the conversation history (chat_history) and use an LLM to generate a search query.

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

We can test this out by passing in an instance where the user asks a follow-up question.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow-up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

We can now test this out end-to-end:

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

We can see that this gives a coherent answer - we've successfully turned our retrieval chain into a chatbot!

### Agent

We've so far created examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

**NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.**

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

1.The retriever we just created. This will let it easily answer questions about LangSmith

2.A search tool. This will let it easily answer questions that require up-to-date information.

First, let's set up a tool for the retriever we just created:

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

The search tool that we will use is Tavily. This will require an API key (they have generous free tier). After creating it on their platform, you need to set it as an environment variable:

In [ ]:
export TAVILY_API_KEY=...

If you do not want to set up an API key, you can skip creating this tool.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

We can now create a list of the tools we want to work with:

In [ ]:
tools = [retriever_tool, search]

Now that we have the tools, we can create an agent to use them. We will go over this pretty quickly - for a deeper dive into what exactly is going on, check out the Agent's Getting Started documentation

Install langchain hub first

In [ ]:
pip install langchainhub

Install the langchain-openai package To interact with OpenAI we need to use langchain-openai which connects with OpenAI SDK[https://github.com/langchain-ai/langchain/tree/master/libs/partners/openai].

In [ ]:
pip install langchain-openai

Now we can use it to get a predefined prompt

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `openai_api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

We can now invoke the agent and see how it responds! We can ask it questions about LangSmith:



In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

We can ask it about the weather:

In [ ]:
agent_executor.invoke({"input": "what is the weather in SF?"})

We can have conversations with it:

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})